## Packages

In [227]:
import numpy as np
import pandas as pd

import statsmodels.api as sm

import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, KFold,cross_val_score
from sklearn.metrics import accuracy_score,mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.inspection import permutation_importance
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor, plot_tree ,export_text

## Data preprocessing

In [ ]:
data = pd.read_csv("BodyFat.csv")
print(data.isnull().sum())
data.describe()

In [ ]:

ls = list(data.loc[:,"BODYFAT"])
ls.sort()
ls

In [ ]:
df = pd.DataFrame(data)
plt.figure(figsize=(10, 5))
df.boxplot()
plt.title('Boxplot of Original Data')
plt.xticks(rotation=75)
plt.show()



### Remove outliers of features

In [ ]:
# using IQR to replace outliers of features with NA's

filtered_columns = [col for col in data.columns if col not in ["IDNO","BODYFAT","DENSITY","AGE"]]
df_removed = df.copy()
for column in filtered_columns:
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    df_removed[column] = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)][column]

# boxplot of fitered data

plt.figure(figsize=(10, 5))
df_removed.boxplot()
plt.xticks(rotation=90)
plt.title('Boxplot of Filtered Data (After Removing Outliers)')
plt.show()

In [ ]:
# check the NA's after removing outliers
df_removed.describe()
X = df_removed.drop(columns=["IDNO","BODYFAT","DENSITY"])
y = df_removed["BODYFAT"]
print("X missing data:\n",X.isnull().sum())
print("y missing data:\n",y.isnull().sum())

### KNN to impute the NA's of features

In [ ]:
# K-fold to search for the best parameter K of KNN method
pipeline = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('imputer', KNNImputer()),  
    ('model',KNeighborsRegressor())  
])

param_grid = {
    'imputer__n_neighbors': [ 3,5,7,9, 11, 13, 15, 17, 19, 21, 25]  
}

kf = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=kf, scoring='neg_mean_squared_error')
grid_search.fit(X, y)

print("Best n_neighbors:", grid_search.best_params_)

### Imputation

In [234]:
imputer = KNNImputer(n_neighbors=3)
df_imputed = pd.DataFrame(imputer.fit_transform(df_removed), columns=df_removed.columns)


In [ ]:
# box plot of imputed data
plt.figure(figsize=(10, 5))
df_imputed.boxplot()
plt.title('Boxplot of Imputed Data')
plt.xticks(rotation=75)
plt.show()

### Dealing with BMI calculating problem

In [ ]:
# box plot of the ratio between theoretical bmi values and the actual bmi values from the dataset
ratio = (df_imputed["WEIGHT"]/df_imputed["HEIGHT"]**2)/df_imputed["ADIPOSITY"]
pd.DataFrame(ratio).describe()
plt.figure(figsize=(10, 5))
pd.DataFrame(ratio).boxplot()
plt.title('Boxplot of Imputed Data')
plt.xticks(rotation=75)
plt.show()

$$ratio = \frac{\frac{Weight}{Height^2}}{bmi}$$

$$1kg = 2.2046lb $$
$$1m = 39.3701in$$
$$\frac{1kg}{1m^2}=\frac{2.2046lb}{(39.3701in)^2}=0.001422\frac{lb}{in^2}$$


In [ ]:
ratio_df = pd.DataFrame(ratio)


outliers_index = []


for col in ratio_df.columns:
    Q1 = ratio_df[col].quantile(0.25)  
    Q3 = ratio_df[col].quantile(0.75)  
    IQR = Q3 - Q1  
    
    lower_bound = Q1 - 1.5 * IQR  
    upper_bound = Q3 + 1.5 * IQR  
    

    outliers = ratio_df[(ratio_df[col] < lower_bound) | (ratio_df[col] > upper_bound)].index
    outliers_index.extend(outliers.tolist()) 

# list the outliers of the ratio values

outliers_index = sorted(set(outliers_index))  
print(f"All Outliers Index: {outliers_index}")

In [ ]:
# use actual bmi values to replace the outliers of weight or height

handle = pd.DataFrame(df_imputed.iloc[outliers_index,:])

mean_height = df_imputed['HEIGHT'].mean()
std_height = df_imputed['HEIGHT'].std()
mean_weight = df_imputed['WEIGHT'].mean()
std_weight =df_imputed['WEIGHT'].std()


handle['zscore_height'] = (handle['HEIGHT'] - mean_height) / std_height
handle['zscore_weight'] = (handle['WEIGHT'] - mean_weight) / std_weight
transform = 0.001422

def recalculate_values(row):
    if abs(row['zscore_height']) > abs(row['zscore_weight']):
        new_height = np.sqrt(row['WEIGHT'] / (row['ADIPOSITY']*transform))
        return new_height, row['WEIGHT']  
    else:
        new_weight = row['ADIPOSITY'] * (row['HEIGHT'] ** 2)*transform
        return row['HEIGHT'], new_weight  

handle = pd.DataFrame(handle)
handle[['new_height', 'new_weight']] = handle.apply(recalculate_values, axis=1, result_type='expand')

handle['HEIGHT'] = handle['new_height']
handle['WEIGHT'] = handle['new_weight']

handle.drop(columns=['zscore_height', 'zscore_weight', 'new_height', 'new_weight'], inplace=True)

df_imputed.set_index('IDNO', inplace=True)
handle.set_index('IDNO', inplace=True)

df_imputed.update(handle)

df_imputed.reset_index(inplace=True)
df_imputed.describe()

In [ ]:
# check the ratio now
ratio = (df_imputed["WEIGHT"]/df_imputed["HEIGHT"]**2)/df_imputed["ADIPOSITY"]

plt.figure(figsize=(10, 5))
pd.DataFrame(ratio).boxplot()
plt.title('Boxplot of Imputed Data')
plt.xticks(rotation=75)
plt.show()

### Handling outliers of Bodyfat

In [ ]:
# use other models to handle outliers of bodyfat

df_imputed["BFP"] =495/(1.0324 - 0.19077*np.log10(df_imputed["ABDOMEN"]-df_imputed["NECK"]) + 0.15456*np.log10(df_imputed["HEIGHT"]*2.54))-450


df_selected = pd.DataFrame(df_imputed[['BFP', 'BODYFAT']])

print(df_selected)

In [ ]:
# visualize the difference between values from dataset and from the adopted model

plt.scatter(df_imputed['BODYFAT'], df_imputed['BFP'], color='blue', label='BFP vs BODYFAT')

plt.title('Comparison of BFP and BODYFAT')
plt.xlabel('BODYFAT')
plt.ylabel('BFP')


plt.plot([df_imputed['BODYFAT'].min(), df_imputed['BODYFAT'].max()], 
         [df_imputed['BODYFAT'].min(), df_imputed['BODYFAT'].max()], 
         color='red', linestyle='--', label='Ideal line (y=x)')

plt.legend()

plt.show()

In [ ]:
# replace the outliers of Bodyfat with results from the adopted model
df_selected['Residual'] = df_selected['BFP'] - df_selected['BODYFAT']


df_selected['Abs_Residual'] = np.abs(df_selected['Residual']/df_selected['BFP'])


df_sorted_by_residual = df_selected.sort_values(by='Abs_Residual', ascending=False)



high_residual_ids = df_sorted_by_residual[df_sorted_by_residual['Abs_Residual'] > 0.5].index


df_imputed.loc[high_residual_ids, 'BODYFAT'] = df_imputed.loc[high_residual_ids, 'BFP']

df_imputed = df_imputed.drop(columns=["BFP"])

df_imputed.describe()

### Scale

In [243]:
# use Minmax method to scale the data and finish the preprocessing work
scaler = MinMaxScaler()
scaler.fit(df_imputed)
df_imputed_scaled = pd.DataFrame(scaler.transform(df_imputed),columns=df_imputed.columns)

## MLR method

### Feature Selection

In [244]:
# forward selection

def forward_selection(X, y, significance_level=0.05):
    initial_features = []
    remaining_features = list(X.columns)
    while remaining_features:
        p_values = pd.Series(index=remaining_features, dtype=float)
        for feature in remaining_features:
            model = sm.OLS(y, sm.add_constant(X[initial_features + [feature]])).fit()
            p_values[feature] = model.pvalues[feature]
        min_p_value = p_values.min()
        if min_p_value < significance_level:
            best_feature = p_values.idxmin()
            initial_features.append(best_feature)
            remaining_features.remove(best_feature)
        else:
            break
    return initial_features

In [245]:
# backward selection
def backward_selection(X, y, significance_level=0.05):
    features = list(X.columns) 
    while len(features) > 0:
        model = sm.OLS(y, X[features]).fit()
        p_values = model.pvalues
        max_p_value = p_values.max()
        if max_p_value > significance_level:
            excluded_feature = p_values.idxmax()
            print(f"Removing {excluded_feature} with p-value {max_p_value}")
            features.remove(excluded_feature)
        else:
            break
    return features

In [246]:
# stepwise selection
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() 
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [247]:
X_imputed_scaled = df_imputed_scaled.drop(columns=["IDNO","BODYFAT","DENSITY"])
y = df_imputed["BODYFAT"]

In [ ]:
# results of forward selection
result_forward = forward_selection(X_imputed_scaled , y)
result_forward

In [ ]:
# results of backward selection
result_backward = backward_selection(X_imputed_scaled , y)
result_backward

In [ ]:
# results of stepwise selection
output_stepwise = stepwise_selection(X_imputed_scaled , y)
result_stepwise = output_stepwise
result_stepwise

### Visualization and cross-validation

In [ ]:
# visualize the result of forward selection

X_forward = X_imputed_scaled.loc[:, result_forward]
X_backward = X_imputed_scaled.loc[:, result_backward]
X_stepwise = X_imputed_scaled.loc[:, result_stepwise] 

model = LinearRegression()

model.fit(X_forward,y)

y_pred = model.predict(X_forward)


plt.figure(figsize=(10, 6))
plt.scatter(y, y_pred, color='blue', label='Predicted vs Actual')
plt.plot([y.min(), y.max()], [y.min(), y.max()], color='red', lw=2, label='Ideal fit')
plt.xlabel('Actual(%)')
plt.ylabel('Predicted(%)')
plt.title('Actual vs Predicted(MLR Forward Selection)')
plt.legend()

plt.show()

# show model MSE
print("Mean Squared Error:", mean_squared_error(y, y_pred))

In [ ]:
# visualize the cross validation result of forward selection
cv_scores = cross_val_score(model, X_forward, y, cv=5, scoring='neg_mean_squared_error')

cv_mse_scores = -cv_scores

print("Cross-Validation MSE for each fold:", cv_mse_scores)

print("Average MSE from Cross-Validation:", cv_mse_scores.mean())

In [ ]:
# residual check of forward selection
residuals = y - y_pred

plt.scatter(y_pred, residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs Predicted Values')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.show()

In [ ]:
# visualize the result of backward selection


model = LinearRegression()

model.fit(X_backward,y)

y_pred = model.predict(X_backward)


plt.figure(figsize=(10, 6))
plt.scatter(y, y_pred, color='blue', label='Predicted vs Actual')
plt.plot([y.min(), y.max()], [y.min(), y.max()], color='red', lw=2, label='Ideal fit')
plt.xlabel('Actual(%)')
plt.ylabel('Predicted(%)')
plt.title('Actual vs Predicted(MLR Backward Selection)')
plt.legend()

plt.show()

# show the mse of backward selection model
print("Mean Squared Error:", mean_squared_error(y, y_pred))

In [ ]:
# show the cross validation result of backward selection
cv_scores = cross_val_score(model, X_backward, y, cv=5, scoring='neg_mean_squared_error')

cv_mse_scores = -cv_scores

print("Cross-Validation MSE for each fold:", cv_mse_scores)

print("Average MSE from Cross-Validation:", cv_mse_scores.mean())

In [ ]:
# check the residuals of backward selection
residuals = y - y_pred

plt.scatter(y_pred, residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs Predicted Values')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.show()

In [ ]:
# visualize the result of stepwise selection

model = LinearRegression()

model.fit(X_stepwise,y)

y_pred = model.predict(X_stepwise)

plt.figure(figsize=(10, 6))
plt.scatter(y, y_pred, color='blue', label='Predicted vs Actual')
plt.plot([y.min(), y.max()], [y.min(), y.max()], color='red', lw=2, label='Ideal fit')
plt.xlabel('Actual(%)')
plt.ylabel('Predicted(%)')
plt.title('Actual vs Predicted(MLR Stepwise Selection)')
plt.legend()

plt.show()


print("Mean Squared Error:", mean_squared_error(y, y_pred))

In [ ]:
# show the cross validation result of stepwise selection

cv_scores = cross_val_score(model, X_stepwise, y, cv=5, scoring='neg_mean_squared_error')

cv_mse_scores = -cv_scores

print("Cross-Validation MSE for each fold:", cv_mse_scores)

print("Average MSE from Cross-Validation:", cv_mse_scores.mean())

In [ ]:
# check the residuals of stepwise selection
residuals = y - y_pred

plt.scatter(y_pred, residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs Predicted Values')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.show()

In [ ]:
# model summary of forward selection

X_with_const = sm.add_constant(X_forward)  
ols_model = sm.OLS(y, X_with_const)
results = ols_model.fit()


print("forward:",results.summary())

In [ ]:
# model summary of backward selection


X_with_const = sm.add_constant(X_backward)  
ols_model = sm.OLS(y, X_with_const)
results = ols_model.fit()


print("backward:",results.summary())

## Decision Tree

In [262]:
# build a decision tree using all the features

X = df_imputed_scaled.drop(columns=["IDNO","BODYFAT","DENSITY"])
y = df_imputed["BODYFAT"]

tree_model = DecisionTreeRegressor(max_depth=None)
tree_model.fit(X, y)

importances = tree_model.feature_importances_

In [ ]:
# list the feature importance 

feature_names = X.columns if isinstance(X, pd.DataFrame) else [f"Feature {i}" for i in range(X.shape[1])]

feature_importance_dict = {feature: importance for feature, importance in zip(feature_names, importances)}


sorted_feature_importance = dict(sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True))


print("Sorted feature importance:")
for feature, importance in sorted_feature_importance.items():
    print(f"{feature}: {importance}")

In [ ]:
# choose the top 6 features

sorted_feature_names = [feature for feature, importance in sorted_feature_importance.items()][:6]

sorted_feature_names

In [ ]:
# use the selected features to build another decision tree model

X_tree_selected = X[sorted_feature_names]

tree_model = DecisionTreeRegressor(max_depth=5)
tree_model.fit(X_tree_selected, y)

y_pred = tree_model.predict(X_tree_selected)


plt.figure(figsize=(10, 6))
plt.scatter(y, y_pred, color='blue', label='Predicted vs Actual')

plt.plot([y.min(), y.max()], [y.min(), y.max()], color='red', lw=2, label='Ideal fit')

plt.xlabel('Actual(%)')
plt.ylabel('Predicted(%)')
plt.title('Actual vs Predicted (Decision Tree Regressor)')
plt.legend()

plt.show()

# show the mse of this decision tree model
print("Mean Squared Error:", mean_squared_error(y, y_pred))

In [ ]:
# cross validation of this decision tree model

cv_scores = cross_val_score(tree_model, X_tree_selected, y, cv=5, scoring='neg_mean_squared_error')

cv_mse_scores = -cv_scores

print("Cross-Validation MSE for each fold:", cv_mse_scores)

print("Average MSE from Cross-Validation:", cv_mse_scores.mean())

In [ ]:
# check the reesiduals of decision tree model

residuals = y - y_pred

plt.scatter(y_pred, residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs Predicted Values')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.show()

In [ ]:
# print the stucture of decision tree model

tree_rules = export_text(tree_model, feature_names=list(X_tree_selected.columns))
print(tree_rules)

## Radar plot of feature importance for each model

In [269]:
# define the radar plot function

def plot_radar(importances, feature_names,method):
    N = len(feature_names)
    angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
    importances = np.concatenate((importances,[importances[0]])) 
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    ax.fill(angles, importances, color='blue', alpha=0.25)
    ax.plot(angles, importances, color='blue', linewidth=2)
    ax.set_yticklabels([])
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(feature_names)

    plt.title(f'Feature Importance Radar Chart for {method}')
    plt.show()



In [ ]:
# radar plot of decision tree
tree_importance = tree_model.feature_importances_
plot_radar(tree_importance, list(X_tree_selected.columns),"Decision Tree")


In [ ]:
# radar plot of forward selection

model_forward = LinearRegression()
model_forward.fit(X_forward, y)


X_forward_with_const = sm.add_constant(X_forward)
forward_sm = sm.OLS(y, X_forward_with_const).fit()

forward_pvalus = forward_sm.pvalues

feature_importance = -np.log(forward_pvalus)


feature_importance = feature_importance.iloc[1:] 
feature_names = X_forward.columns  


plot_radar(feature_importance, feature_names, "P-value based Importance for Forward")



In [ ]:
# radar plot of backward selection

model_backward = LinearRegression()
model_backward.fit(X_backward, y)


X_backward_with_const = sm.add_constant(X_backward)
backward_sm = sm.OLS(y, X_backward_with_const).fit()

backward_pvalus = backward_sm.pvalues

feature_importance = -np.log(backward_pvalus)


feature_importance = feature_importance.iloc[1:] 
feature_names = X_backward.columns  


plot_radar(feature_importance, feature_names, "P-value based Importance for Backward")

In [ ]:
# radar plot of stepwise selection

model_stepwise = LinearRegression()
model_stepwise.fit(X_stepwise, y)


X_stepwise_with_const = sm.add_constant(X_stepwise)
stepwise_sm = sm.OLS(y, X_stepwise_with_const).fit()

stepwise_pvalus = stepwise_sm.pvalues

feature_importance = -np.log(stepwise_pvalus)


feature_importance = feature_importance.iloc[1:] 
feature_names = X_stepwise.columns  


plot_radar(feature_importance, feature_names, "P-value based Importance for Stepwise")
